In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import random

class Musteri:
    def __init__(self, musteri_numarasi, ad, soyad):
        self.musteri_numarasi = musteri_numarasi
        self.ad = ad
        self.soyad = soyad

class Hesap:
    def __init__(self, hesap_numarasi, musteri, bakiye=0):
        self.hesap_numarasi = hesap_numarasi
        self.musteri = musteri
        self.bakiye = bakiye
        self.acik_hesap = True

    def para_yatirmak(self, miktar):
        if self.acik_hesap and miktar > 0:
            self.bakiye += miktar
            self.guncelle_hesap()
            return f"Yatırılan Miktar: {miktar}\nGüncel Bakiye: {self.bakiye}"
        elif not self.acik_hesap:
            return "Hesap Kapalı."
        else:
            return "Geçersiz Miktar."

    def para_cekmek(self, miktar):
        if self.acik_hesap and miktar > 0 and miktar <= self.bakiye:
            self.bakiye -= miktar
            self.guncelle_hesap()
            return f"Çekilen Miktar: {miktar}\nGüncel Bakiye: {self.bakiye}"
        elif not self.acik_hesap:
            return "Hesap Kapalı."
        elif miktar > self.bakiye:
            return "Bakiye Yetersiz."
        else:
            return "Geçersiz Miktar."

    def bakiye_sorgulama(self):
        if self.acik_hesap:
            return f"Mevcut Bakiye: {self.bakiye}"
        else:
            return "Hesap Kapalı."

    def hesap_kapatmak(self):
        if self.acik_hesap:
            self.acik_hesap = False
            self.guncelle_hesap()
            return "Hesap Kapatıldı."
        else:
            return "Hesap Zaten Kapalı."

    def hesap_aktif_etmek(self):
        if not self.acik_hesap:
            self.acik_hesap = True
            self.guncelle_hesap()
            return "Hesap Aktif Edildi."
        else:
            return "Hesap Zaten Aktif."

    def guncelle_hesap(self):
        connection = sqlite3.connect("banka.db")
        cursor = connection.cursor()

        cursor.execute("UPDATE Hesaplar SET bakiye = ?, acik_hesap = ? WHERE hesap_numarasi = ?",
                       (self.bakiye, 1 if self.acik_hesap else 0, self.hesap_numarasi))

        connection.commit()
        connection.close()

class Banka:
    def __init__(self):
        self.baglanti_olustur()

    def baglanti_olustur(self):
        self.connection = sqlite3.connect("banka.db")
        self.cursor = self.connection.cursor()

        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Musteriler (
                        musteri_numarasi VARCHAR(11) PRIMARY KEY,
                        ad VARCHAR(100),
                        soyad VARCHAR(100)
                        )''')

        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Hesaplar (
                        hesap_numarasi INTEGER PRIMARY KEY,
                        musteri_numarasi VARCHAR(11),
                        bakiye REAL,
                        acik_hesap BIT,
                        FOREIGN KEY (musteri_numarasi) REFERENCES Musteriler(musteri_numarasi)
                        )''')

        self.connection.commit()

    def baglanti_kapat(self):
        self.connection.close()

    def hesap_olusturmak(self, musteri, ilk_bakiye=0):
        hesap_numarasi = random.randint(1000,99999999)
        hesap = Hesap(hesap_numarasi, musteri, ilk_bakiye)
        self.kaydet_musteri(musteri)
        self.kaydet_hesap(hesap)
        return f"{hesap_numarasi} numaralı hesap oluşturuldu."

    def kaydet_musteri(self, musteri):
        self.cursor.execute("INSERT INTO Musteriler VALUES (?, ?, ?)", (musteri.musteri_numarasi, musteri.ad, musteri.soyad))
        self.connection.commit()

    def kaydet_hesap(self, hesap):
        self.cursor.execute("INSERT INTO Hesaplar VALUES (?, ?, ?, ?)",
                            (hesap.hesap_numarasi, hesap.musteri.musteri_numarasi, hesap.bakiye, 1 if hesap.acik_hesap else 0))
        self.connection.commit()

    def tum_hesaplar(self):
        print("\nTüm Hesaplar: ")
        self.cursor.execute("SELECT * FROM Hesaplar")
        hesap_rows = self.cursor.fetchall()

        for row in hesap_rows:
            hesap_numarasi, musteri_numarasi, bakiye, acik_hesap = row

            musteri = self.musteri_getir(musteri_numarasi)
            hesap = Hesap(hesap_numarasi, musteri, bakiye)
            hesap.acik_hesap = True if acik_hesap == 1 else False

            hesap.bilgileri_goster()

    def tum_musteriler(self):
        print("\nTüm Müşteriler: ")
        self.cursor.execute("SELECT * FROM Musteriler")
        musteri_rows = self.cursor.fetchall()

        for row in musteri_rows:
            musteri_numarasi, ad, soyad = row
            musteri = Musteri(musteri_numarasi, ad, soyad)
            musteri.bilgileri_goster()

    def hesap_numarasi_kontrolu(self, hesap_numarasi):
        self.cursor.execute("SELECT * FROM Hesaplar WHERE hesap_numarasi = ?", (hesap_numarasi,))
        hesap_row = self.cursor.fetchone()

        if hesap_row:
            musteri = self.musteri_getir(hesap_row[1])
            hesap = Hesap(hesap_row[0], musteri, hesap_row[2])
            hesap.acik_hesap = True if hesap_row[3] == 1 else False
            return hesap
        else:
            return None

    def musteri_getir(self, musteri_numarasi):
        self.cursor.execute("SELECT * FROM Musteriler WHERE musteri_numarasi = ?", (musteri_numarasi,))
        musteri_row = self.cursor.fetchone()

        if musteri_row:
            return Musteri(musteri_row[0], musteri_row[1], musteri_row[2])
        else:
            return None

banka = Banka()

def hesap_olustur():
    musteri_numarasi = musteri_numarasi_entry.get()
    ad = ad_entry.get()
    soyad = soyad_entry.get()

    if musteri_numarasi and ad and soyad:
        musteri = Musteri(musteri_numarasi, ad, soyad)
        sonuc = banka.hesap_olusturmak(musteri)
        messagebox.showinfo("Sonuç", sonuc)
    else:
        messagebox.showwarning("Uyarı", "Lütfen tüm alanları doldurunuz.")

def para_yatir():
    hesap_numarasi = hesap_numarasi_entry.get()
    miktar = miktar_entry.get()

    if hesap_numarasi and miktar:
        try:
            hesap_numarasi = int(hesap_numarasi)
            miktar = float(miktar)
            hesap = banka.hesap_numarasi_kontrolu(hesap_numarasi)
            if hesap:
                sonuc = hesap.para_yatirmak(miktar)
                messagebox.showinfo("Sonuç", sonuc)
            else:
                messagebox.showwarning("Uyarı", "Hesap Bulunamadı.")
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen geçerli bir miktar giriniz.")
    else:
        messagebox.showwarning("Uyarı", "Lütfen tüm alanları doldurunuz.")

def para_cek():
    hesap_numarasi = hesap_numarasi_entry.get()
    miktar = miktar_entry.get()

    if hesap_numarasi and miktar:
        try:
            hesap_numarasi = int(hesap_numarasi)
            miktar = float(miktar)
            hesap = banka.hesap_numarasi_kontrolu(hesap_numarasi)
            if hesap:
                sonuc = hesap.para_cekmek(miktar)
                messagebox.showinfo("Sonuç", sonuc)
            else:
                messagebox.showwarning("Uyarı", "Hesap Bulunamadı.")
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen geçerli bir miktar giriniz.")
    else:
        messagebox.showwarning("Uyarı", "Lütfen tüm alanları doldurunuz.")

def bakiye_sorgula():
    hesap_numarasi = hesap_numarasi_entry.get()

    if hesap_numarasi:
        try:
            hesap_numarasi = int(hesap_numarasi)
            hesap = banka.hesap_numarasi_kontrolu(hesap_numarasi)
            if hesap:
                sonuc = hesap.bakiye_sorgulama()
                messagebox.showinfo("Sonuç", sonuc)
            else:
                messagebox.showwarning("Uyarı", "Hesap Bulunamadı.")
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen geçerli bir hesap numarası giriniz.")
    else:
        messagebox.showwarning("Uyarı", "Lütfen hesap numarasını giriniz.")

def hesap_kapat():
    hesap_numarasi = hesap_numarasi_entry.get()

    if hesap_numarasi:
        try:
            hesap_numarasi = int(hesap_numarasi)
            hesap = banka.hesap_numarasi_kontrolu(hesap_numarasi)
            if hesap:
                sonuc = hesap.hesap_kapatmak()
                messagebox.showinfo("Sonuç", sonuc)
            else:
                messagebox.showwarning("Uyarı", "Hesap Bulunamadı.")
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen geçerli bir hesap numarası giriniz.")
    else:
        messagebox.showwarning("Uyarı", "Lütfen hesap numarasını giriniz.")

def hesap_aktif_et():
    hesap_numarasi = hesap_numarasi_entry.get()

    if hesap_numarasi:
        try:
            hesap_numarasi = int(hesap_numarasi)
            hesap = banka.hesap_numarasi_kontrolu(hesap_numarasi)
            if hesap:
                sonuc = hesap.hesap_aktif_etmek()
                messagebox.showinfo("Sonuç", sonuc)
            else:
                messagebox.showwarning("Uyarı", "Hesap Bulunamadı.")
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen geçerli bir hesap numarası giriniz.")
    else:
        messagebox.showwarning("Uyarı", "Lütfen hesap numarasını giriniz.")

def tum_hesaplari_listele():
    connection = sqlite3.connect("banka.db")
    cursor = connection.cursor()

    cursor.execute("SELECT * FROM Hesaplar")
    hesaplar = cursor.fetchall()

    if hesaplar:
        hesap_listesi = ""
        for hesap in hesaplar:
            hesap_listesi += f"Hesap Numarası: {hesap[0]}, Müşteri Numarası: {hesap[1]}, Bakiye: {hesap[2]}, Açık Hesap: {'Evet' if hesap[3] else 'Hayır'}\n"
        messagebox.showinfo("Tüm Hesaplar", hesap_listesi)
    else:
        messagebox.showinfo("Bilgi", "Hiçbir hesap bulunamadı.")

    connection.close()

    

# Tkinter GUI oluşturma
app = tk.Tk()
app.title("Banka Yönetim Sistemi")

tk.Label(app, text="Müşteri Numarası:").grid(row=0, column=0)
musteri_numarasi_entry = tk.Entry(app)
musteri_numarasi_entry.grid(row=0, column=1)

tk.Label(app, text="Ad:").grid(row=1, column=0)
ad_entry = tk.Entry(app)
ad_entry.grid(row=1, column=1)

tk.Label(app, text="Soyad:").grid(row=2, column=0)
soyad_entry = tk.Entry(app)
soyad_entry.grid(row=2, column=1)

tk.Label(app, text="Hesap Numarası:").grid(row=3, column=0)
hesap_numarasi_entry = tk.Entry(app)
hesap_numarasi_entry.grid(row=3, column=1)

tk.Label(app, text="Miktar:").grid(row=4, column=0)
miktar_entry = tk.Entry(app)
miktar_entry.grid(row=4, column=1)


tk.Button(app, text="Hesap Oluştur", command=hesap_olustur).grid(row=5, column=0)
tk.Button(app, text="Para Yatır", command=para_yatir).grid(row=5, column=1)
tk.Button(app, text="Para Çek", command=para_cek).grid(row=6, column=0)
tk.Button(app, text="Bakiye Sorgula", command=bakiye_sorgula).grid(row=6, column=1)
tk.Button(app, text="Hesap Kapat", command=hesap_kapat).grid(row=7, column=0)
tk.Button(app, text="Hesap Aktif Et", command=hesap_aktif_et).grid(row=7, column=1)
tk.Button(app, text="Tüm Hesapları Listele", command=tum_hesaplari_listele).grid(row=8, column=0, columnspan=2)

app.mainloop()